In [1]:
import spectHR as cs
import pyhrv

In [2]:
DataSet = cs.SpectHRDataset("Example Data/SUB_002.xdf")

In [3]:
DataSet = cs.borderData(DataSet)
#DataSet.ecg = DataSet.ecg.slicetime(500, 515)

In [4]:
DataSet = cs.filterECGData(DataSet, {"filterType": "highpass", "cutoff": .50})

In [5]:
DataSet = cs.calcPeaks(DataSet)

In [6]:
%matplotlib widget
GUI = cs.prepPlot(DataSet, 500, 600)

In [7]:
display(GUI)

AppLayout(children=(HBox(children=(Select(class_='ma-2', color='primary', items=['Drag', 'Add', 'Find', 'Remov…

In [8]:
cs.handler.show_logs()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [9]:
print(DataSet.RTops)

             time ID       ibi
0      411.122864  N  0.621008
1      411.743872  N  0.603584
2      412.347456  N  0.602682
3      412.950139  N  0.588900
4      413.539039  N  0.585735
...           ... ..       ...
2583  2352.170497  N  0.654699
2584  2352.825196  N  0.628791
2585  2353.453988  N  0.641538
2586  2354.095526  N  0.678079
2587  2354.773605  N       NaN

[2588 rows x 3 columns]
